Project 3 Part 1

Senait Abate

In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
import os, time, json, math
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [7]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [8]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [9]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [10]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)

ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)

akas = pd.read_csv(akas_url,sep='\t', low_memory=False)

Basics Cleaning

In [11]:
#reviewing data and checking for missing values
print(basics.info(), '\n')
print(basics.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9188391 entries, 0 to 9188390
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 630.9+ MB
None 

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64


In [12]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [13]:
#Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [14]:
#confirming 6695269 missing values in runtimeMinutes column
basics[basics['runtimeMinutes'].isnull()]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
18,tt0000019,short,The Clown Barber,The Clown Barber,0,1898,NaN,NaN,"Comedy,Short"
22,tt0000024,short,Opening of the Kiel Canal,Opening of the Kiel Canal,0,1895,NaN,NaN,"News,Short"
23,tt0000025,short,The Oxford and Cambridge University Boat Race,The Oxford and Cambridge University Boat Race,0,1895,NaN,NaN,"News,Short,Sport"
36,tt0000038,short,The Ball Game,The Ball Game,0,1898,NaN,NaN,"Documentary,Short,Sport"
37,tt0000039,short,Barnet Horse Fair,Barnet Horse Fair,0,1896,NaN,NaN,Short
...,...,...,...,...,...,...,...,...,...
9188384,tt9916844,tvEpisode,Episode #3.15,Episode #3.15,0,2010,NaN,NaN,"Action,Drama,Family"
9188385,tt9916846,tvEpisode,Episode #3.18,Episode #3.18,0,2010,NaN,NaN,"Action,Drama,Family"
9188386,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,NaN,NaN,"Action,Drama,Family"
9188387,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,NaN,NaN,"Action,Drama,Family"


In [15]:
#Eliminate movies that are null for runtimeMinutes
basics = basics.dropna(axis=0, subset=['runtimeMinutes'])

In [16]:
basics[basics['genres'].isnull()]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
498,tt0000502,movie,Bohemios,Bohemios,0,1905,NaN,100,NaN
1616,tt0001630,movie,Der fremde Vogel,Der fremde Vogel,0,1911,NaN,45,NaN
1739,tt0001756,movie,Lucha por la herencia,Lucha por la herencia,0,1911,NaN,92,NaN
2069,tt0002094,movie,Charles IV,Charles IV,0,1912,NaN,50,NaN
2289,tt0002315,movie,El lobo de la sierra,El lobo de la sierra,0,1912,NaN,76,NaN
...,...,...,...,...,...,...,...,...,...
9187007,tt9913834,movie,Hiniku: Nure nawazeme,Hiniku: Nure nawazeme,0,1981,NaN,64,NaN
9187028,tt9913878,movie,Document porno: Yubi ijime,Document porno: Yubi ijime,0,1981,NaN,61,NaN
9187603,tt9915130,movie,Meet John Doe,Meet John Doe,0,NaN,NaN,120,NaN
9188046,tt9916134,movie,The Thing in the Bag,The Thing in the Bag,0,NaN,NaN,78,NaN


In [17]:
#Eliminate movies that are null for genres
basics = basics.dropna(axis=0, subset=['genres'])

In [18]:
#reviewing subtypes of titleType column
basics['titleType'].value_counts()

tvEpisode       1064241
short            574550
movie            367163
video            175133
tvMovie           88323
tvSeries          86592
tvSpecial         16470
tvMiniSeries      16082
tvShort            9380
videoGame           298
Name: titleType, dtype: int64

In [19]:
#keep only titleType==Movie
basics = basics.drop(basics[basics['titleType'] != 'movie'].index)

In [20]:
#confirming only movie subtype in titleType column
basics['titleType'].value_counts()

movie    367163
Name: titleType, dtype: int64

In [21]:
#checking subtypes for startYear
basics['startYear'].value_counts()

2017    14193
2018    14139
2016    13825
2019    13811
2015    13329
        ...  
1894        1
1903        1
1899        1
1904        1
1896        1
Name: startYear, Length: 129, dtype: int64

In [22]:
#drop all values less than 2000
basics = basics.drop(basics[basics['startYear'] <= '1999'].index)

In [23]:
#confirm startYear 2000-2022
basics['startYear'].value_counts()

2017    14193
2018    14139
2016    13825
2019    13811
2015    13329
2014    12983
2013    12274
2021    11737
2012    11543
2020    11257
2011    10685
2010    10124
2009     9271
2008     8070
2022     7791
2007     6884
2006     6431
2005     5760
2004     5133
2003     4528
2002     4088
2001     3813
2000     3594
2023      324
2024       30
2025        8
2026        2
2027        1
Name: startYear, dtype: int64

In [24]:
#exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [25]:
#confirm documentary not in genres column
starts_doc = basics['genres'].str.startswith('doc', 'Doc')

In [26]:
basics[starts_doc]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


Akas Cleaning

In [27]:
#Replace "\N" with np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [28]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [29]:
#checking value counts for region column to confirm abbv are countries
akas['region'].value_counts()

FR    3943552
JP    3943487
DE    3926631
IN    3871875
ES    3866627
       ...   
TV          1
NU          1
PW          1
NR          1
TC          1
Name: region, Length: 247, dtype: int64

In [30]:
#review US count in region column
akas.loc[akas['region'] == 'US']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
33026613,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
33026650,tt9916720,10,The Demonic Nun,US,NaN,tv,NaN,0
33026652,tt9916720,12,The Nun 2,US,NaN,imdbDisplay,NaN,0
33026669,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


In [31]:
#drop all values less than 2000
akas = akas.drop(akas[akas['region'] != 'US'].index)

In [32]:
#confirm only US region
akas['region'].value_counts()

US    1345187
Name: region, dtype: int64

Ratings Cleaning

In [33]:
#reviewing data and checking for missing values
print(ratings.info(), '\n')
print(ratings.isna().sum())

#no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258095 entries, 0 to 1258094
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1258095 non-null  object 
 1   averageRating  1258095 non-null  float64
 2   numVotes       1258095 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB
None 

tconst           0
averageRating    0
numVotes         0
dtype: int64


Filter basics df based on akas df

In [34]:
#filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34790       True
61090       True
67636       True
77930      False
78417      False
           ...  
9188063     True
9188072     True
9188111    False
9188156     True
9188240    False
Name: tconst, Length: 147107, dtype: bool

In [35]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61090,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67636,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86767,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
92732,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
9187527,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9187923,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9188063,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9188072,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


Create data folder

In [36]:
#save current basics dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [37]:
#open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016.0,NaN,74,Comedy


In [38]:
#save current ratings dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [39]:
#open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1907
1,tt0000002,5.8,256
2,tt0000003,6.5,1706
3,tt0000004,5.6,168
4,tt0000005,6.2,2520


In [40]:
#save current akas dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [41]:
#open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [86]:
#Install tmdbsimple
!pip install tmdbsimple

In [87]:
#load api credentials
import json
with open('/Users/abate/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
#display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [88]:
#set api key to api credential
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

Query movies by revenue, budget, and certification.

In [89]:
#example query of the matrix as a movie object
movie = tmdb.Movies(603)

In [90]:
#check info for movie example
info = movie.info()
info

#budget and revenue included in movie info but not certification

{'adult': False,
 'backdrop_path': '/n2nm4aZRmXyJ9LT4xQX9X6ThcP7.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 86.514,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [91]:
#can search with IMDB_ID
#example with budget of Tom and Jerry
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']

50000000

In [92]:
#how to get certification rating example if movie is in the US

response = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

PG
PG
PG


In [93]:
# Get the movie object for the current id
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']

In [94]:
#create a function that adds certification to dictionary results for movie.info()

def get_movie_with_rating(movie_id):
    #get movie and release dates
    movie = tmdb.Movies(movie_id)
    #construct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
#test function with The Avengers
test = get_movie_with_rating("tt0848228")
test

#has budget, revenue, and certification info

In [ ]:
#test function with The Notebook
test2 = get_movie_with_rating("tt0332280")
test2

#has budget, revenue, and certification info

In [96]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [ ]:
#define years 2000 and 2001
YEARS_TO_GET = [2000, 2001]

In [ ]:
#outer loop
from tqdm.notebook import tqdm_notebook
import time
#set up progress bar to keep track of work
for YEAR in tqdm_notebook (YEARS_TO_GET,desc='YEAR',position=0):time.sleep(0.5)

In [107]:
#define JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

In [108]:
# Check if file exists
file_exists = os.path.isfile(JSON_FILE)

In [109]:
# create file and save empty dictionary with imdb_id
# If it does not exist: create it
if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

In [112]:
# define/ filter IDs to call
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

In [113]:
#divide data by year
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst'].copy()

In [114]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

In [115]:
# filter out any ids that are already in the JSON_FILE
# movie_ids_to_get variable
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [116]:
#inner loop

#function to append new results to the existing JSON file
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [118]:
#add certification to .info results

#Get index and movie id from list
# INNER Loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
    try:
        temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
    except Exception as e:
        continue

Movies from 2001:   0%|          | 0/1529 [00:00<?, ?it/s]

In [119]:
#Save the year's results as csv.gz file
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)